In [1]:
%matplotlib widget
%load_ext autoreload
%autoreload 2

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as GridSpec
import numpy as np
import pandas as pd

import os


import math


from scipy.optimize import curve_fit




from JJformulas import *

In [2]:
import sys 
sys.path.append('..\measDC\_jjtools')

# from Exps import *

from JJ_data_processing import read_opj_data, pbi, bpbi, show_df, update_df
from JJ_data_processing import xy_by_id,extract_Isw_R0_by_id


from JJformulas import *
from meas_util import *

from JJmeasurement import *

D:\anaconda\lib\site-packages\qcodes\dataset\database.py:13: UserWarning: The module `qcodes.dataset.database` is deprecated.
Public features are available at the import of `qcodes`.
Private features are available in `qcodes.dataset.sqlite.*` modules.
  warnings.warn('The module `qcodes.dataset.database` is deprecated.\n'
D:\anaconda\lib\site-packages\tqdm\autonotebook\__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


# Samples

In [3]:
jj70N5 = JJmeas(sample = 'D070B1N5v1')

jj70N5.db_connect()

In [4]:
jj70N1 = JJmeas(sample = 'D070B1N1v1')

jj70N1.db_connect()

In [5]:
jj70N0 = JJmeas(sample = 'D070B1N0v1')

jj70N0.db_connect()

In [6]:

jj70N7 = JJmeas(sample = 'D070B1N7')

jj70N7.db_connect()

# Some coding

In [7]:
from scipy import interpolate

fig, ax = plt.subplots(figsize = (4,5))

def gaussian(x, mu, sig):
    return 1./(np.sqrt(2.*np.pi)*sig)*np.exp(-np.power((x - mu)/sig, 2.)/2)

Rsh = 377

kT = 0.050
Ec = 1
EJ = 0.4
Vc_noise = np.sqrt(2*Ec*kT)/1e4


Npts = 501
V_ = np.linspace(-1e-3, 1e-3, Npts)/1e1


def IZ_conv(V_, EJ, Vc_noise, kT, Rsh):
    
    sigma = Vc_noise#np.sqrt(Ec*kT)/1e4
    Is = I_IZ( V_, EJ = EJ, R = Rsh, T = kT)

    Vs = V_ - Is*Rsh

    f = interpolate.interp1d(Vs, Is)




    Vs_evengrid = np.linspace(np.min(Vs), np.max(Vs), len(Vs))
    
    dVs =  (np.max(Vs) - np.min(Vs)) / (len(Vs)-1)
    
    

    Is_evengrid = f(Vs_evengrid)

    Is = Is_evengrid
    Vs = Vs_evengrid

    V_c = gaussian(Vs, 0, sigma)*dVs

    Is_conv = np.convolve(Is, V_c, 'same')
#     print(len(Is))
    
    return Vs, Is, Is_conv

Vs, Is, Is_conv = IZ_conv(V_, EJ, Vc_noise, kT, Rsh)


ax.plot(Vs/1e-6, Is/1e-9, 'o', label = 'cIZ')

ax.plot(Vs/1e-6 ,Is_conv/1e-9, label = 'cIZ + Vnoise')



ax.set_xlabel('V, uV')
ax.set_ylabel('I, nA')

ax.set_title('EJ = {:g} K, T = {:g} K, Rb = {:g} Ohm'.format(EJ, kT, Rsh))

# ax.set_xlim(0, 300)
# ax.set_ylim(0, .0250)

ax.legend()

plt.tight_layout()

FigureCanvasNbAgg()

In [14]:


Rsh = 377

kT = 0.05
# Ec = 10
Vc_noise = 50e-6

EJ_list = np.logspace(-2,1.5, 51)

Npts = 251
V_ = np.linspace(-1e-3, 1e-3, Npts)

Isws = []
Isws_conv = []

R0s = []
R0s_conv = []

for EJ in EJ_list:
    Vs, Is, Is_conv = IZ_conv(V_, EJ, Vc_noise , kT, Rsh)
    
#     Isw = (np.max(Is) - np.min(Is))/2
#     Isw_conv = (np.max(Is_conv) - np.min(Is_conv))/2
    
    Imax = np.max(Is)
    Imax_conv = np.max(Is_conv)

    Vmax = Vs[np.argmax(Is)]
    Vmax_conv = Vs[np.argmax(Is_conv)]
    
    R0s.append( Vmax/Imax )
    R0s_conv.append( Vmax_conv/Imax_conv )
    
    
    
    Isws.append(Imax)
    Isws_conv.append(Imax_conv)
    

    


In [22]:
ft = pd.read_excel("lit_summary.xlsx")
# ft = pd.read_excel("exp summary.xlsx")
ft

,id,Rn_tot,Isw,Ro_tot,N,Rn,Ro,Ec,Ej,Iab,comment,ref,Ic_AB,Ej_alt
0,Jack PhD p43,47.9,1.5n,13.3k,1.0,NaN,13.3k,1.0,540.0m,NaN,unknown Ec,Jack phd,22.7n,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Jack PhD p54 1,NaN,200.0p,100.0k,1.0,NaN,100.0k,NaN,220.0m,NaN,unknown Ec,Jack phd,9.2n,NaN
3,Jack PhD p54 2,NaN,50.0p,400.0k,1.0,NaN,400.0k,NaN,100.0m,NaN,unknown Ec,Jack phd,4.2n,NaN
4,Jack PhD p54 3,NaN,15.0p,1.3M,1.0,NaN,1.3M,NaN,60.0m,NaN,unknown Ec,Jack phd,2.5n,NaN
5,Jack PhD p54 4,NaN,2.0p,10.0M,1.0,NaN,10.0M,NaN,20.0m,NaN,unknown Ec,Jack phd,839.0p,NaN
6,Jack PhD p54 5,NaN,500.0f,40.0M,1.0,NaN,40.0M,NaN,10.0m,NaN,unknown Ec,Jack phd,419.5p,NaN
7,Jack PhD p56 0,NaN,6.6n,3.0k,1.0,NaN,800.0,NaN,1.2,NaN,unknown Ec,Jack phd,113.3n,NaN
8,Jack PhD p56 1,NaN,15.0n,1.3k,1.0,NaN,1.3k,NaN,1.9,NaN,unknown Ec,Jack phd,79.7n,NaN


# Isw vs Ej

In [10]:
# ax = ft.plot( x = 'EJ', y = 'Isw', color ='k', marker = 'D', ls = '')
# ax.set_xscale('log')
# ax.set_yscale('log')

In [32]:
# ax = ft.plot( x = 'EJ', y = 'Isw', color ='k', marker = 'D', ls = '')


fig, ax = plt.subplots(figsize = (10,8))
markers=['d',',','o','v','^','<','>','s','p','P','*','h','H','+','x','X','D','d','|','_','1','2','3','4','8']

unique_lablist = list(set(ft.ref.to_list()))


for ej, isw, lab, m in zip(ft.Ej, ft.Isw, ft.ref, markers):
    
    ilab = unique_lablist.index(lab)
    ax.scatter(ej, isw, color = f'C{ilab}',marker = m, label = lab, s = 50)
    
# ax.legend()

ax.set_xscale('log')
ax.set_yscale('log')

ax.set_xlim(8e-3, 2e1)
ax.set_ylim(1e-13, 1e-6)

FigureCanvasNbAgg()

(1e-13, 1e-06)

In [33]:
# fig, ax = plt.subplots()


ax.plot(EJ_list,  EJ_list*kB/hbar*2*e, '--', label = '$A-B$')


ax.plot(EJ_list, Isws, label = 'cIZ')
ax.plot(EJ_list, Isws_conv, label = 'cIZ + Vc_noise')


ax.set_xscale('log')
ax.set_yscale('log')

leg1 = ax.legend(loc='lower right')

In [50]:
jj70N5.db_connect()

B_list = np.linspace(-.4e-3, .2e-3, 121)
jj70N5.ZF   = -0.093e-3
jj70N5.FF   = 0.130e-3
ids = range(689,809+1)


# B_list = np.linspace(0e-3, .2e-3, 51)
# jj70N5.ZF   = 0e-3
# jj70N5.FF   = 0.220e-3
# ids = range(1213, 1263+1)

cos_list = abs (jj70N5.B_to_cos(B_list))



Ics = [2*extract_Isw_R0_by_id (idx, dy = 50e-6)[0] for idx in ids]

Ej0 = 0.410

l70N5v1, = ax.plot(Ej0*cos_list, Ics, '.', fillstyle = 'none')

e:\OneDrive - Rutgers University\files_GitHub\JJ_arrays\JJ_data_processing.py:285: RuntimeWarning: invalid value encountered in less
  ind1 = np.where(np.abs(vA0) < dx )


In [51]:
jj70N5.db_connect()

# B_list = np.linspace(-.4e-3, .2e-3, 121)
# jj70N5.ZF   = -0.093e-3
# jj70N5.FF   = 0.130e-3
# ids = range(689,809+1)


B_list = np.linspace(0e-3, .2e-3, 51)
jj70N5.ZF   = 0e-3
jj70N5.FF   = 0.220e-3
ids = range(1213, 1263+1)

cos_list = abs (jj70N5.B_to_cos(B_list))



Ics = [2*extract_Isw_R0_by_id (idx, dy = 50e-6)[0] for idx in ids]

Ej0 = 0.410

l70N5v2, = ax.plot(Ej0*cos_list, Ics, '.', fillstyle = 'none')

e:\OneDrive - Rutgers University\files_GitHub\JJ_arrays\JJ_data_processing.py:285: RuntimeWarning: invalid value encountered in less
  ind1 = np.where(np.abs(vA0) < dx )


In [52]:
jj70N1.db_connect()

jj70N1.ZF = -2e-6
jj70N1.FF = 200e-6

B_list = np.linspace(-.10e-3, .5e-3, 121)
cos_list = abs (jj70N1.B_to_cos(B_list))

ids = range(14, 134+1)

Ics = [2*extract_Isw_R0_by_id (idx, dy = 30e-6)[0] for idx in ids]

Ej0 = 0.350
l70N1, = ax.plot(Ej0*cos_list, Ics, '.', fillstyle = 'none' )

e:\OneDrive - Rutgers University\files_GitHub\JJ_arrays\JJ_data_processing.py:285: RuntimeWarning: invalid value encountered in less
  ind1 = np.where(np.abs(vA0) < dx )


In [53]:
jj70N0.db_connect()

jj70N0.ZF = -2e-6
jj70N0.FF = 200e-6


# B_list = np.linspace(-.10e-3, .5e-3, 61)
# cos_list = abs (jj70N0.B_to_cos(B_list))
# ids = range(11, 71+1)

sl = slice(75, 125)
B_list = np.linspace(-1e-3, 1e-3, 201)[sl]
cos_list = abs (jj70N0.B_to_cos(B_list))
ids = range(76, 276+1)[sl]



Ics = [2*extract_Isw_R0_by_id (idx, dy = 50e-6)[0] for idx in ids]

Ej0 = 0.656
l70N0, = ax.plot(Ej0*cos_list, Ics, '.', fillstyle = 'none')

e:\OneDrive - Rutgers University\files_GitHub\JJ_arrays\JJ_data_processing.py:285: RuntimeWarning: invalid value encountered in less
  ind1 = np.where(np.abs(vA0) < dx )


In [54]:
# fig, ax = plt.subplots()
    
jj70N7.db_connect()

jj70N7.ZF = 4e-6
jj70N7.FF = 224e-6


B_list = np.linspace(4e-6, 254e-6, 85)
cos_list = abs (jj70N7.B_to_cos(B_list))

ids = range(208, 292+1)



Ics = [1*extract_Isw_R0_by_id (idx, dy = 50e-6, yoff = -.735e-3)[0] for idx in ids]

Ej0 = 1.18
l70N7, = ax.plot(Ej0*cos_list, Ics, '.', fillstyle = 'none')

D:\anaconda\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
D:\anaconda\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


len is zero
len is zero
len is zero
len is zero
len is zero
len is zero
len is zero
len is zero


In [55]:
jj79 = JJmeas(sample = 'D079N5')

jj79.db_connect()

cos_list_79 = np.linspace(1, 0, 51 )
Ejs_79 = 4.14*cos_list_79


ids = range(72,122+1)

Ics = [2*extract_Isw_R0_by_id (idx, dy = 30e-6)[0] for idx in ids]
    

Ej0 = 4.14
l79N5, = ax.plot(Ej0*cos_list_79, Ics, '.', fillstyle = 'none')

D:\anaconda\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
D:\anaconda\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


len is zero
len is zero


In [56]:
leg2 = ax.legend([l70N5v1,l70N5v2, l70N1, l70N0, l70N7,l79N5],
                 ['70N5v1, N=14, Ec = 650mK',
                  '70N5v2, N=14, Ec = 650mK',
                  '70N1, N=15, Ec = ??mK', 
                  '70N0, N=15, Ec = 400mK', 
                  '70N7, N=14, Ec = 200mK', 
                  '79N5, N=1, Ec = 4mK'], 
                 loc='upper left')
# Manually add the first legend back
ax.add_artist(leg1)

# sandbox

In [251]:
jj70N7.ZF = -.002e-3
jj70N7.FF = .220e-3

jj70N7.cos_to_B(0.0)

0.00021999999999999998

In [226]:
fig, ax = plt.subplots()
    
jj70N7.db_connect()

jj70N7.ZF = 4e-6
jj70N7.FF = 224e-6


B_list = np.linspace(4e-6, 254e-6, 85)
cos_list = abs (jj70N7.B_to_cos(B_list))

ids = range(208, 292+1)



Ics = [1*extract_Isw_R0_by_id (idx, dy = 50e-6, yoff = -.735e-3)[0] for idx in ids]

Ej0 = 1.18
ax.plot(Ej0*cos_list, Ics, '.', fillstyle = 'none')

D:\anaconda\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


FigureCanvasNbAgg()

D:\anaconda\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
D:\anaconda\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


len is zero
len is zero
len is zero
len is zero
len is zero
len is zero
len is zero
len is zero


In [215]:
jj = jj70N0
jj70N0.db_connect()

jj.ZF = -0e-6
jj.FF = 210e-6

sl = slice(75, 125)

B_list = np.linspace(-1e-3, 1e-3, 201)[sl]
cos_list = abs (jj.B_to_cos(B_list))

ids = range(76, 276+1)[sl]

Ics = [2e12*extract_Isw_R0_by_id (idx, dy = 50e-6)[0] for idx in ids]


fig, ax = plt.subplots()
ax.plot(cos_list, Ics, 'o-')
# ax.plot(B_list, Ics, 'o-')

# ax.set_xscale('log')
# ax.set_xlim(0.1, 1)
# ax.set_yscale('log')

e:\OneDrive - Rutgers University\files_GitHub\JJ_arrays\JJ_data_processing.py:285: RuntimeWarning: invalid value encountered in less
  ind1 = np.where(np.abs(vA0) < dx )
D:\anaconda\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


FigureCanvasNbAgg()

In [242]:
fig, ax = plt.subplots()

jj70N5.db_connect()

B_list = np.linspace(-.4e-3, .2e-3, 121)
jj70N5.ZF   = -0.093e-3
jj70N5.FF   = 0.130e-3
ids = range(689,809+1)


# B_list = np.linspace(0e-3, .2e-3, 51)
# jj70N5.ZF   = 0e-3
# jj70N5.FF   = 0.220e-3
# ids = range(1213, 1263+1)

cos_list = abs (jj70N5.B_to_cos(B_list))



Ics = [2*extract_Isw_R0_by_id (idx, dy = 50e-6)[0] for idx in ids]

Ej0 = 0.410

ax.plot(Ej0*cos_list, Ics, '.', fillstyle = 'none', label = '70N5 v1')

D:\anaconda\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


FigureCanvasNbAgg()

e:\OneDrive - Rutgers University\files_GitHub\JJ_arrays\JJ_data_processing.py:285: RuntimeWarning: invalid value encountered in less
  ind1 = np.where(np.abs(vA0) < dx )


In [243]:
jj70N5.db_connect()

# B_list = np.linspace(-.4e-3, .2e-3, 121)
# jj70N5.ZF   = -0.093e-3
# jj70N5.FF   = 0.130e-3
# ids = range(689,809+1)


B_list = np.linspace(0e-3, .2e-3, 51)
jj70N5.ZF   = 0e-3
jj70N5.FF   = 0.220e-3
ids = range(1213, 1263+1)

cos_list = abs (jj70N5.B_to_cos(B_list))



Ics = [2*extract_Isw_R0_by_id (idx, dy = 50e-6)[0] for idx in ids]

Ej0 = 0.34

ax.plot(Ej0*cos_list, Ics, '.', fillstyle = 'none', label = '70N5 v2')

ax.legend()

ax.set_xscale('log')
ax.set_yscale('log')

ax.set_xlim(0.05, .5)

e:\OneDrive - Rutgers University\files_GitHub\JJ_arrays\JJ_data_processing.py:285: RuntimeWarning: invalid value encountered in less
  ind1 = np.where(np.abs(vA0) < dx )


(0.05, 0.5)